In [186]:
import pandas as pd
import numpy as np
import math

INPUT

Option 1: Generate random weight using randomWeightAndBiosGeneratorForNodes

In [187]:
#first = input layer, last = output layer, all remainings = hidden layers
allLayers = {1:[1,2,3], 2:[4, 5], 3: [6,7], 4:[8,9,10], 5:[11]}
#allLayers = {1:[1,2,3], 2:[4, 5], 3: [6]}
learning_rate = .9 #eta
y = 12

import random
#generates random weight for each node specified in multi layer data structure
def randomWeightAndBiosGeneratorForNodes(layers):
  w = {}
  b = {}
  if(len(layers) <= 1):
    return w
  for i in range(len(layers)):
    current = layers[i+1]
    next = {}
    if(i+2 in layers):
      next = layers[i+2]
    for currNode in current:
      b[currNode] = round(random.uniform(1, 100), 2)
      for nextNode in next:
        index =  int(str(currNode) + str(nextNode)) #link between two nodes
        w[index] = round(random.uniform(1, 100), 2)
  return w, b
weight, bios = randomWeightAndBiosGeneratorForNodes(allLayers)

#generates random features(/input) for input layer specified in multi layer data structure
def randomFeatureGenerator(inputLayer):
  x = {}
  for i in inputLayer:
    x[i] = round(random.uniform(1, 100), 2)
  return x
x = randomFeatureGenerator(allLayers[1])


Option 2: Provide all input manually

In [189]:

allLayers = {1:[1,2,3], 2:[4, 5], 3: [6]}
x = {1:1, 2:0, 3:1}
y = 1
weight = {14:0.2, 15:-0.3, 24:0.4, 25:0.1, 34:-0.5, 35:0.2, 46:-0.3, 56:-0.2}
bios = {4:-0.4, 5: 0.2, 6:0.1} #theta
learning_rate = .9 #eta

In [190]:
print("allLayers:", allLayers)
print("weight", weight)
print("bios/theta:", bios)
print("x/input/features:", x)
print("y/output/label:", y)
print("learning rate/eta:", learning_rate)



allLayers: {1: [1, 2, 3], 2: [4, 5], 3: [6]}
weight {14: 0.2, 15: -0.3, 24: 0.4, 25: 0.1, 34: -0.5, 35: 0.2, 46: -0.3, 56: -0.2}
bios/theta: {4: -0.4, 5: 0.2, 6: 0.1}
x/input/features: {1: 1, 2: 0, 3: 1}
y/output/label: 1
learning rate/eta: 0.9


Step 1: Forward Propogration (Caclucate Z and A value)

In [191]:
#calcuates z value (part of forward propogration) for given node, weight, features/x values and bios
def forwardProp(node, w, f, b):
  z = b
  print("\n---node:", node)
  for i in range(len(w)):
    print("w:", w[i], " x:", f[i], " bios/theta:", b)
    z = z + (w[i] * f[i])
  print("z is ", z)
  return z

#calcuates sigmoid for z value
def sigmoid(z):
    a = (1/(1 + math.exp(-z)))
    print(z, a)
    return a

def calculateZandA(layers, w, b, x):
  z = {}
  a = {}
  nextX = x
  for i in range(len(layers)):
    current = layers[i+1] #starts with 0+1 = 1 = [1,2,3]
    tempX = {}
    next = {}
    if(i+2 in layers): #starts with 0+2 = 2 = [4,5]
      next = layers[i+2]
    for nextNode in next:
      nxtNodeWeights = []
      for currNode in current:
        index =  int(str(currNode) + str(nextNode)) #link between two nodes
        if index in w:
          nxtNodeWeights.append(w[index])
      z[nextNode] = forwardProp(nextNode, nxtNodeWeights, list(nextX.values()), bios[nextNode])
      a[nextNode] = sigmoid(z[nextNode])
      tempX[nextNode] = a[nextNode]
    #set features/x for next layer
    nextX = tempX
  return z, a

z,a = calculateZandA(allLayers, weight, bios, x)
print("\n\nSTEP 1: Forward Propogration Result:")
print("z:", z)
print("a:", a)


---node: 4
w: 0.2  x: 1  bios/theta: -0.4
w: 0.4  x: 0  bios/theta: -0.4
w: -0.5  x: 1  bios/theta: -0.4
z is  -0.7
-0.7 0.3318122278318339

---node: 5
w: -0.3  x: 1  bios/theta: 0.2
w: 0.1  x: 0  bios/theta: 0.2
w: 0.2  x: 1  bios/theta: 0.2
z is  0.10000000000000003
0.10000000000000003 0.52497918747894

---node: 6
w: -0.3  x: 0.3318122278318339  bios/theta: 0.1
w: -0.2  x: 0.52497918747894  bios/theta: 0.1
z is  -0.10453950584533817
-0.10453950584533817 0.47388889882398544


STEP 1: Forward Propogration Result:
z: {4: -0.7, 5: 0.10000000000000003, 6: -0.10453950584533817}
a: {4: 0.3318122278318339, 5: 0.52497918747894, 6: 0.47388889882398544}


Step 2.1: Backward Propogration: Find Error Value

In [192]:
def findError(layers, w, a, y):
  lengh = len(layers)
  err = {}
  for i in range(lengh):
    if(i == lengh - 1):
      return err
    
    current = layers[lengh-i] #starts with 3-0 = 3 = [3]
    
    next = current
    if(lengh-i+1 in layers): #starts with 3-0+1 = 4 (not exist so next = current) = [6]
      next = layers[lengh-i+1]
    
    for currNode in current:
      sumWeightAndError = 0
      for nextNode in next:
        index =  int(str(currNode) + str(nextNode)) #link between two nodes
        if index in w and nextNode in err:
          sumOfWeightAndError = sumWeightAndError + (w[index] * err[nextNode])
       #print(index)
      #totalOfErrorsWeight
      if(i == 0): #for output layers
        err[currNode] = a[currNode] * (1 - a[currNode]) * (y - a[currNode])
      else: #for hidden layers
        err[currNode] = a[currNode] * (1 - a[currNode]) * sumOfWeightAndError
  return err
errors = findError(allLayers, weight, a, y)
print("\n\nBackward Propogration Find Error result:")
print("Error:", errors)




Backward Propogration Find Error result:
Error: {6: 0.1311690782143445, 4: -0.008724561965433263, 5: -0.006542085064168994}


Step 2.2: Backward Propogration: Calculate/Update Weights and Bios depending on error

In [194]:
#backward propogration: calculate/Update Weights and Bios depending on error
def backwordPropgrationUpdateWeights(layers, w, l, err, a):
  nWeight = {}
  if(len(layers) <= 1):
    return w
  for i in range(len(layers)):
    current = layers[i+1]
    next = {}
    if(i+2 in layers):
      next = layers[i+2]
    for currNode in current:
      for nextNode in next:
        index =  int(str(currNode) + str(nextNode)) #link between two nodes
        #print(index)
        if(nextNode not in err):
            err[nextNode] = 0
        if(currNode not in a):
            a[currNode] = 0
        nWeight[index] = w[index] + (l * err[nextNode] * a[currNode]) 
  return nWeight
new_weight = backwordPropgrationUpdateWeights(allLayers, weight, learning_rate, errors, a)
print("updated weight:", new_weight)

def backwordPropgrationUpdateBios(b, l, err):
  nBios = {}
  for i in b:
    nBios[i] = b[i] + (l * err[i])
  return x

new_bios = backwordPropgrationUpdateBios(bios, learning_rate, errors)
print("updated bios/theta:", new_bios)

updated weight: {14: 0.2, 15: -0.3, 24: 0.4, 25: 0.1, 34: -0.5, 35: 0.2, 46: -0.26082884634154524, 56: -0.13802506750700472}
updated bios/theta: {1: 1, 2: 0, 3: 1}
